<div style="border-radius:10px;
            border: #0b0265 solid;
           background-color:#e8efff;
           font-size:110%;
           letter-spacing:0.5px;
            text-align: center">

<center><h1 style="padding: 25px 0px; color:#0b0265; font-weight: bold; font-family: Cursive">
Santander customer transaction prediction</h1></center>
<center><h3 style="padding-bottom: 25px; color:#0b0265; font-weight: bold; font-style:italic; font-family: Cursive">
(With LGBMClassifier)</h3></center>     

</div>

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from lightgbm import LGBMClassifier
from sklearn.metrics import auc, roc_curve, roc_auc_score

# Define Functions

In [ ]:
kfold = StratifiedKFold(n_splits=5, random_state=14, shuffle=True)

def RndSrch_Tune(model, X, y, params):
    
    clf = RandomizedSearchCV(model, params, scoring ='roc_auc', cv = kfold, n_jobs=-1, random_state=100)
    clf.fit(X, y)
    print("best score is :" , clf.best_score_)
    print("best estimator is :" , clf.best_estimator_)
    print("best Params is :" , clf.best_params_)
    return (clf.best_score_)
# --------------------------------------------------------

def cross_validation(x, y, model):
    result= cross_val_score(model, x, y, cv=kfold, scoring="roc_auc", n_jobs=-1)
    print("Score: %f" % result.mean())

# Data Understanding & Preproccessing

In [ ]:
train = pd.read_csv("/kaggle/input/santander-customer-transaction-prediction/train.csv")
test = pd.read_csv("/kaggle/input/santander-customer-transaction-prediction/test.csv")
train.head()

In [ ]:
train.info()

In [ ]:
sizes = train['target'].value_counts(sort = True)
colors = ["orange","lightgreen"] 
rcParams['figure.figsize'] = 5,5 
plt.pie(sizes, explode = (0,0.1), labels=["0", "1"], colors=colors,
        autopct='%.2f%%', shadow=True, startangle=90)
plt.show()

In [ ]:
train.drop('ID_code', axis=1, inplace=True)

In [ ]:
train[train.columns[train.isna().sum() > 0]].isna().mean()*100
train.shape

In [ ]:
train.drop_duplicates(inplace=True)
train.shape

In [ ]:
x=train.drop(['target'], axis=1)
y=train['target']

In [ ]:
scaler=MinMaxScaler()
x_train=pd.DataFrame(scaler.fit_transform(x), columns=x.columns)
x_train.head()

# Feature Selection

In [ ]:
modelSelect_LGB = LGBMClassifier(random_state=14, n_jobs=-1, class_weight='balanced')
modelSelect_LGB.fit(x_train, y)
importance_LGB = modelSelect_LGB.feature_importances_
for i,v in enumerate(importance_LGB):
    print('Feature: %0d, Score: %.6f' % (i, v))

In [ ]:
list_features_LGB = []
for i,v in enumerate(importance_LGB):
    if(v > 0.008):
        list_features_LGB.append(i)

print("The best features are : ", list_features_LGB)
print("Number of best features: ", len(list_features_LGB))

In [ ]:
x_select_LGB = x_train.iloc[:, list_features_LGB]
x_select_LGB.head()

# Modeling with tuned hyperparameters

<div style="padding: 5px 0px; font-family: Cursive; font-size:16px; background-color:#eaeefc;padding: 25px 10px">
We used Random search for tuning hyperparameters</div>

In [ ]:
# lgb = LGBMClassifier(**{'reg_lambda': 0.1, 
#                         'reg_alpha': 0.2, 
#                         'num_leaves': 50, 
#                          'n_estimators': 250, 
#                         'min_child_samples': 700, 
#                          'max_bin': 1000, 
#                         'learning_rate': 0.1})

# scores = cross_validation(x_select_LGB, y, lgb)
# print(scores)

In [ ]:
# Value of hyperparameters for random search

# param_lgb = {'learning_rate':[0.2,0.1,0.01,0.05,0.001],
#              'num_leaves':range(10,100,10),
#              'min_child_samples':range(500,1000,100),
#              'reg_alpha':[0.1,0.01,0.2,0.3],
#              'reg_lambda':[0.1,0.01,0.2,0.3],
#             'n_estimators':range(50,300,50),
#              'max_bin': range(500,1500,100)}

# RndSrch_Tune(LGBMClassifier(random_state = 14, n_jobs=-1), x_select_LGB, y, param_lgb)

In [ ]:
lgb = LGBMClassifier(**{'reg_lambda': 0.3, 
                          'reg_alpha': 0.01, 
                          'num_leaves': 20, 
                          'n_estimators': 250, 
                          'min_child_samples': 600, 
                          'learning_rate': 0.05,
                          'objective': 'binary',
                          'n_jobs': -1,
                          'class_weight':'balanced',
                          'random_state':14})

scores = cross_validation(x_select_LGB, y, lgb)
print(scores)

# Prediction & Submission

In [ ]:
tmp_test = test.iloc[:,1:]
tmp_test.head()

In [ ]:
test.drop_duplicates(inplace=True)
x_test=pd.DataFrame(scaler.transform(tmp_test), columns=tmp_test.columns)
x_select_test = x_test.iloc[:, list_features_LGB]
x_select_test.head()

In [ ]:
lgb.fit(x_select_LGB, y)
y_pred_XGB = lgb.predict(x_select_LGB)
y_pred_XGB_test = lgb.predict(x_select_test)

In [ ]:
output = pd.DataFrame({'ID_code': test.ID_code, 
                       'target': y_pred_XGB_test})
output.head()

In [ ]:
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

<div style="padding: 5px 0px; font-family: Cursive; font-size:16px; background-color:#eaeefc;padding: 25px 10px">
Thank you very much, <a href="https://www.kaggle.com/ehsandahesh">Ehsan</a> for your cooperation. <br> You can see our other notebook related to this competition which is with algorithm XGBClassifier <a href="https://www.kaggle.com/code/ehsandahesh/santander-customer-transaction-prediction-xgbc">here</a>.</div>

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style:solid;
            border-color: #0b0265;
            letter-spacing:0.5px;">

<center><h4 style="padding: 5px 0px; color:#0b0265; font-weight: bold; font-family: Cursive">
    Thanks for your attention and for reviewing our notebook.🙌 <br><br>Please write your comments for us.📝</h4></center>
<center><h4 style="padding: 5px 0px; color:#0b0265; font-weight: bold; font-family: Cursive">
If you liked our work and found it useful, please upvote. Thank you🙏</h4></center>
</div>